In [8]:
import pandas as pd

import plotly.express as px
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
# (Segmentación y modelo Experto Paula)

In [9]:
df = pd.read_csv("C:/Users/rabad/Desktop/PFB/Cosas/Partes/datos_prestamos_limpio.csv")
pd.set_option("display.max_columns", None)
df

,id,loan_amnt,funded_amnt,int_rate,installment,grade,annual_inc,dti,fico_range_high,fico_range_low,revol_util,loan_status,out_prncp,total_rec_prncp,total_rec_int,last_fico_range_high,last_fico_range_low,open_acc,pub_rec,inq_last_6mths,home_ownership,recoveries,num_tl_90g_dpd_24m,purpose,delinq_2yrs,term_months,emp_length_years
0,68407277,3600.0,3600.0,13.99,123.03,C,55000.0,5.910000,679.0,675.0,29.700001,Fully Paid,0.00,3600.00,821.72,564.0,560.0,7.0,0.0,1.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0,36,10+
1,68355089,24700.0,24700.0,11.99,820.28,C,65000.0,16.059999,719.0,715.0,19.200001,Fully Paid,0.00,24700.00,979.66,699.0,695.0,22.0,0.0,4.0,MORTGAGE,0.0,0.0,small_business,1.0,36,10+
2,68341763,20000.0,20000.0,10.78,432.66,B,63000.0,10.780000,699.0,695.0,56.200001,Fully Paid,0.00,20000.00,2705.92,704.0,700.0,6.0,0.0,0.0,MORTGAGE,0.0,0.0,home_improvement,0.0,60,10+
3,66310712,35000.0,35000.0,14.85,829.90,C,110000.0,17.059999,789.0,785.0,11.600000,Current,15897.65,19102.35,12361.66,679.0,675.0,13.0,0.0,0.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0,60,10+
4,68476807,10400.0,10400.0,22.45,289.91,F,104433.0,25.370001,699.0,695.0,64.500000,Fully Paid,0.00,10400.00,1340.50,704.0,700.0,12.0,0.0,3.0,MORTGAGE,0.0,0.0,major_purchase,1.0,60,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260663,89885898,24000.0,24000.0,12.79,543.50,C,95000.0,19.610001,669.0,665.0,84.400002,Current,14287.39,9712.61,6047.89,709.0,705.0,5.0,0.0,0.0,MORTGAGE,0.0,0.0,home_improvement,0.0,60,7
2260664,88977788,24000.0,24000.0,10.49,515.74,B,108000.0,34.939999,699.0,695.0,39.000000,Current,13951.42,10048.58,4959.95,734.0,730.0,24.0,1.0,1.0,MORTGAGE,0.0,0.0,debt_consolidation,0.0,60,10+
2260665,88985880,40000.0,40000.0,10.49,859.56,B,227000.0,12.750000,709.0,705.0,64.900002,Current,23252.59,16747.41,8156.52,724.0,720.0,5.0,0.0,1.0,MORTGAGE,0.0,7.0,debt_consolidation,7.0,60,9
2260666,88224441,24000.0,24000.0,14.49,564.56,C,110000.0,18.299999,664.0,660.0,68.099998,Charged Off,0.00,3521.91,3233.49,594.0,590.0,10.0,1.0,0.0,RENT,0.0,0.0,debt_consolidation,0.0,60,6


In [10]:
estados_loan_status = df["loan_status"].unique()
print(estados_loan_status)

['Fully Paid' 'Current' 'Charged Off' 'In Grace Period'
 'Late (31-120 days)' 'Late (16-30 days)' 'Default'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']


In [11]:
# He decidido segmentar por los distintos estados de nuestro target loan_status

df_fully_paid = df[df["loan_status"] == "Fully Paid"]
df_current = df[df["loan_status"] == "Current"]
df_charged_off = df[df["loan_status"] == "Charged Off"]
df_grace_period = df[df["loan_status"] == "In Grace Period"]
df_late_31_120 = df[df["loan_status"] == "Late (31-120 days)"]
df_late_16_30 = df[df["loan_status"] == "Late (16-30 days)"]
df_default = df[df["loan_status"] == "Default"]

In [12]:
df["pagado"] = df["loan_status"].apply(lambda x: 1 if x == "Fully Paid" else 0) # Me he decantado por usar una columna binaria para el entrenamiento, usando "pagado"

X = df[["loan_amnt", "int_rate", "annual_inc", "fico_range_high"]]  # Características que tengo en cuenta para el modelo
y = df["pagado"]  # Variable target

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42)

modelo = LogisticRegression(max_iter = 1000)
modelo.fit(X_train, y_train)

prediccion = modelo.predict(X_test)

print(classification_report(y_test, prediccion))

              precision    recall  f1-score   support

           0       0.56      0.66      0.61    237033
           1       0.53      0.43      0.48    215101

    accuracy                           0.55    452134
   macro avg       0.55      0.55      0.54    452134
weighted avg       0.55      0.55      0.55    452134



In [ ]:
# Matriz de Confusión para la visualización de resultados

matriz_conf = confusion_matrix(y_test, prediccion)
df_matriz_conf = pd.DataFrame(matriz_conf, index=["No Pagado", "Pagado"], columns=["No Pagado", "Pagado"])

fig = px.imshow(df_matriz_conf, 
                text_auto = True, 
                color_continuous_scale = "Blues",
                title = "Matriz de Confusión",
                labels = dict(x = "Predicción", y = "Actual"),
                x = ["No Pagado", "Pagado"], 
                y = ["No Pagado", "Pagado"])
fig.show()

In [ ]:
# Curva ROC para ver el rendimiento del modelo

fpr, tpr, thresholds = roc_curve(y_test, modelo.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

df_curva_roc = pd.DataFrame({"FPR": fpr, "TPR": tpr})

fig = px.line(df_curva_roc,
              x = "FPR",
              y = "TPR",
              title = f"Curva ROC (AUC = {roc_auc:.2f})",
              labels = {"FPR": "Tasa de Falsos Positivos", "TPR": "Tasa de Verdaderos Positivos"})

fig.add_scatter(x = [0, 1],
                y = [0, 1],
                mode = "lines",
                line = dict(dash = "dash",
                            color = "red"),
                            name = "Línea Aleatoria")
fig.show()

In [ ]:
# (Fine tuning César)

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
# Hiperparametros para el fine tuning
param_grid = {
    "penalty": ["l1", "l2", "elasticnet", "none"],
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "solver": ["saga", "liblinear"],
    "class_weight": [None, "balanced"]
}

# Ajuste del GridSearchCV
grid_search = GridSearchCV(estimator = modelo,
                           param_grid = param_grid,
                           cv = 5,
                           scoring = "roc_auc",
                           verbose = 1,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:") # Los mejores hiperparámetros
print(grid_search.best_params_)

# Predicciones utilizando el mejor modelo
mejor_modelo = grid_search.best_estimator_
prediccion_grid = mejor_modelo.predict(X_test)

# Para ver el classification report
print(classification_report(y_test, prediccion_grid))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 

In [ ]:
# Matriz de Confusión del GridSearchCV
matriz_conf = confusion_matrix(y_test, prediccion_grid)
df_matriz_conf = pd.DataFrame(matriz_conf, index = ["No Pagado", "Pagado"], columns = ["No Pagado", "Pagado"])

fig = px.imshow(df_matriz_conf, 
                text_auto = True, 
                color_continuous_scale = "Blues",
                title = "Matriz de Confusión",
                labels = dict(x = "Predicción", y = "Actual"),
                x = ["No Pagado", "Pagado"], 
                y = ["No Pagado", "Pagado"])
fig.show()

In [ ]:
# Curva ROC del GridSearchCV
fpr, tpr, thresholds = roc_curve(y_test,
                                 mejor_modelo.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

df_curva_roc = pd.DataFrame({"FPR": fpr,
                             "TPR": tpr})

fig = px.line(df_curva_roc,
              x = "FPR",
              y = "TPR",
              title = f"Curva ROC (AUC = {roc_auc:.2f})",
              labels = {"FPR": "Tasa de Falsos Positivos", "TPR": "Tasa de Verdaderos Positivos"})

fig.add_scatter(x = [0, 1],
                y = [0, 1],
                mode = "lines",
                line = dict(dash = "dash",
                            color = "red"),
                name = "Línea Aleatoria")
fig.show()